In [2]:
#https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.names

data_source = "https://goo.gl/vhm1eU"
name = ['prg', 'plas', 'pres','skin','test','mass','pedi', 'age','class']

In [3]:
import csv
import requests
import numpy as np

with requests.Session() as s:
    download = s.get(data_source)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    data = np.array(list(cr), dtype=np.float32)

In [5]:
import tensorflow as tf
from sklearn.preprocessing import normalize

x_data = data[:,:-1]
y_data = data[:,-1]

normalize_x = normalize(x_data, axis=0)

x = np.insert(normalize_x, 0 , 1, axis=1)
print(x)
x = x.T

[[ 1.          0.04235974  0.04270833 ...,  0.03679789  0.03924821
   0.05117289]
 [ 1.          0.00705996  0.02452844 ...,  0.02913166  0.02197149
   0.03172719]
 [ 1.          0.05647966  0.05280828 ...,  0.02551758  0.04206507
   0.03275065]
 ..., 
 [ 1.          0.03529979  0.03491695 ...,  0.02869359  0.01533622
   0.03070373]
 [ 1.          0.00705996  0.0363598  ...,  0.03296478  0.02184629
   0.04810252]
 [ 1.          0.00705996  0.026837   ...,  0.03329333  0.019718
   0.02353953]]


In [4]:
W = tf.Variable(tf.random_uniform([1,len(x)], -1.0, 1.0))

X = tf.placeholder(tf.float32, shape=[len(x),None])
Y = tf.placeholder(tf.float32)

h = tf.matmul(W, X)
hypothesis = tf.sigmoid(h)
#hypothesis = tf.div(1., 1. + tf.exp(-h))

cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))

W_val = []
cost_val = []

a = tf.Variable(0.01)
optimizer = tf.train.GradientDescentOptimizer(a) # 라이브러리
train = optimizer.minimize(cost)

In [5]:
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for step in range(20000000):
    sess.run(train, feed_dict={X:x, Y:y_data})
    if step % 400000 == 0:
        print (step, sess.run(cost, feed_dict={X:x, Y:y_data}), sess.run(W))

0 0.648238 [[-0.64370704  0.8196727  -0.06390308  0.71320814 -0.78145862 -0.83034396
   0.02984565 -0.2788853  -0.46313614]]
400000 0.594924 [[-1.72449946  8.10843372  7.3362174   1.41440964  1.34788287  4.1775713
   4.36387634  5.14467525  3.92126036]]
800000 0.571735 [[ -2.44696021  12.15105247  13.60021305   1.55449331   1.86450291
    5.67106438   8.01581573   8.38665009   6.92604828]]
1200000 0.557784 [[ -2.9613924   14.33965397  19.10077477   1.40177989   1.86335838
    5.9303546   11.29950237  10.4531517    9.07270241]]
1600000 0.547438 [[ -3.36438084  15.47490025  24.22650719   1.07850659   1.70288622
    5.75578737  14.35126019  11.80105209  10.68858624]]
2000000 0.539248 [[ -3.68944502  16.          28.80414391   0.65301353   1.51215136
    5.37431765  17.27497101  12.71005154  11.89767361]]
2400000 0.532165 [[ -3.96583676  16.          33.38177872   0.15881999   1.3214165
    4.99284792  19.75351715  13.33020782  12.9128828 ]]
2800000 0.525793 [[ -4.21877575  16.          37

In [6]:
data_x = np.insert(normalize_x, 0 , 1, axis=1)

W_val = sess.run(W).reshape(len(x),1)
y_data = y_data.reshape(len(y_data),1)

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

x_w = np.dot(data_x, W_val)

sum((sigmoid(x_w) > 0.5) == y_data)/ y_data.shape[0]

array([ 0.7734375])